##### **This notebook verifies we have enough data for user level reply coordination**

In [11]:
df[['tweetid', 'userid', 'tweet_text', 'in_reply_to_userid']] .head()

,tweetid,userid,tweet_text,in_reply_to_userid
79,1272844638005735425,827246726021005314,@kiando_ اوه دهنم سرویس تر شد که 😖😖,1.143906e+18
82,1247717737155215361,827246726021005314,@Adisia5 انتخابت درستِ ✊🏼✊🏼🤤,1.223365e+18
83,1213896951483097089,827246726021005314,@kermeabrisham2 ابجو میخورم میرم شمال,1.138879e+18
93,1236505668053590016,827246726021005314,@itspersephoneh 👏🏼👏🏼👌🏼,1.157649e+18
112,1339061299745964033,1170591537081671680,@_llemonade Hadeeh,1.278112e+18


In [3]:
import pandas as pd
import numpy as np
import warnings
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
from tqdm import tqdm
import sys
import os

import importlib

#### packages
import helper.strategy_helper as st
import helper.visualization as vz
import helper.helper as hp
import helper.file_helper as file_hp
import config.config as config

In [4]:
config = config.config()
path = config['PATHS']

derived_path = path['derived_path']
all_tweet_data = path['all_tweet_data']
plot_path = path['plot_path']

file_hp.create_folder(plot_path, 'reply_count')
reply_path = os.path.join(plot_path, 'reply_count')

In [5]:
all_campaigns, names = st.bundle_campaign()

In [11]:
def test_count(path, external=True):
    type_of = 'ops'
    info =[]

    for row in all_campaigns:
        for year in row:
            for new_campaign in row[year]:

                print(f'\n ------START: {year}: {new_campaign} ------- \n')
#                 test = input("Do you wanna test : 1/0 ")

#                 if test == '0':
#                     continue

                data_path = st.get_data_path(all_tweet_data, year, 
                                     new_campaign)

                data = st.read_ops_control_data(data_path['ops'],
                                                data_path['control'], 
                                             [type_of])
                df_w = data[type_of]

                if 'tweet_language' not in df_w.columns:
                    continue

                # df_test = df_w.loc[df_w['tweet_language'] == 'en']
                
                df = df_w.loc[~df_w['in_reply_to_tweetid'].isnull()]
                
                if external == True:
                    df = st.reply_to_external_users(df)

                print('Eng lang replies ', len(df))
                df_grps = (df.groupby(['in_reply_to_userid',
                                       # 'in_reply_to_tweetid'
                                      ])['userid'] #.apply(lambda x: list(np.unique(x)))
                              .size()
                              .to_frame('count_replies')
                              .reset_index())
                df_grps_size = df_grps.loc[df_grps['count_replies']>=5]
                total = len(df_grps_size)
                
                print('Greater than equal to 5 :', 
                      total)
                
                if total == 0:
                    min_val = 0
                    max_val = 0
                else:
                    min_val = min(df_grps_size['count_replies'])
                    max_val = max(df_grps_size['count_replies'])

                info.append([year,
                             new_campaign,
                             total,
                             min_val,
                             max_val
                            ])
                
                if len(df_grps_size) == 0:
                    continue

                xlabel = 'Number of users replying to each tweet'
                ylabel = 'Raw count of tweets'

                print('Min no. of reply ', 
                      min(df_grps_size['count_replies']))
                print('Max no. of reply ', 
                      max(df_grps_size['count_replies']))

                # plot_histogram(df_grps_size, xlabel, ylabel,
                #                reply_path, f'{year}_{new_campaign}'
                #               )

#                 val = input("Enter your value: ")

#                 print(val)

   

    (pd.DataFrame(data=info,
                  columns=['year', 'campaign', 'size',
                           'min','max'
                          ]
                 )
     .to_csv(f'{path}')
    )

In [12]:
filename ='english_external_user_level_reply_info.csv'

path = os.path.join(derived_path, filename)

test_count(path)


 ------START: 2021_12: CNHU_0621 ------- 

Eng lang replies  1183
Greater than equal to 5 : 31
Min no. of reply  5
Max no. of reply  50

 ------START: 2021_12: CNCC_0621 ------- 

Eng lang replies  13212
Greater than equal to 5 : 557
Min no. of reply  5
Max no. of reply  452

 ------START: 2021_12: MX_0621 ------- 

Eng lang replies  5577
Greater than equal to 5 : 93
Min no. of reply  5
Max no. of reply  1470

 ------START: 2021_12: REA_0621 ------- 

Eng lang replies  878
Greater than equal to 5 : 3
Min no. of reply  54
Max no. of reply  643

 ------START: 2021_12: RNA_0621 ------- 

Eng lang replies  3871
Greater than equal to 5 : 95
Min no. of reply  5
Max no. of reply  228

 ------START: 2021_12: Tanzania_0621 ------- 

Eng lang replies  12781
Greater than equal to 5 : 186
Min no. of reply  5
Max no. of reply  2010

 ------START: 2021_12: uganda_0621 ------- 

Eng lang replies  137205
Greater than equal to 5 : 3694
Min no. of reply  5
Max no. of reply  8083

 ------START: 2021_12:

In [16]:
# print(path)
# df = pd.read_csv(path)
# print(df.info())

In [ ]:
## how many times a user was replied to 

In [17]:
def test_how_often_user_replied_to_other_user(path, external=True):
    type_of = 'ops'
    info =[]

    for row in all_campaigns:
        for year in row:
            for new_campaign in row[year]:

                print(f'\n ------START: {year}: {new_campaign} ------- \n')
#                 test = input("Do you wanna test : 1/0 ")

#                 if test == '0':
#                     continue

                data_path = st.get_data_path(all_tweet_data, year, 
                                     new_campaign)

                data = st.read_ops_control_data(data_path['ops'],
                                                data_path['control'], 
                                             [type_of])
                df_w = data[type_of]

                if 'tweet_language' not in df_w.columns:
                    continue

                # df_test = df_w.loc[df_w['tweet_language'] == 'en']
                
                df = df_w.loc[~df_w['in_reply_to_tweetid'].isnull()]
                
                if external == True:
                    df = st.reply_to_external_users(df)

                print('Eng lang replies ', len(df))
                df_grps = (df.groupby(['in_reply_to_userid',
                                       # 'in_reply_to_tweetid'
                                       'userid'
                                      ])['in_reply_to_tweetid']
                              .size()
                              .to_frame('count_replies')
                              .reset_index())
                df_grps_size = df_grps.loc[df_grps['count_replies']>=5]
                total = len(df_grps_size)
                
                print('Greater than equal to 5 :', 
                      total)
                
                if total == 0:
                    min_val = 0
                    max_val = 0
                else:
                    min_val = min(df_grps_size['count_replies'])
                    max_val = max(df_grps_size['count_replies'])

                info.append([year,
                             new_campaign,
                             total,
                             min_val,
                             max_val
                            ])
                
                if len(df_grps_size) == 0:
                    continue

                xlabel = 'Number of users replying to each tweet'
                ylabel = 'Raw count of tweets'

                print('Min no. of reply ', 
                      min(df_grps_size['count_replies']))
                print('Max no. of reply ', 
                      max(df_grps_size['count_replies']))

                # plot_histogram(df_grps_size, xlabel, ylabel,
                #                reply_path, f'{year}_{new_campaign}'
                #               )

#                 val = input("Enter your value: ")

#                 print(val)

   

    (pd.DataFrame(data=info,
                  columns=['year', 'campaign', 'size',
                           'min','max'
                          ]
                 )
     .to_csv(f'{path}')
    )

In [18]:
filename ='user_to_user_tweet_count_reply_info.csv'

path = os.path.join(derived_path, filename)

test_how_often_user_replied_to_other_user(path)


 ------START: 2021_12: CNHU_0621 ------- 

Eng lang replies  1183
Greater than equal to 5 : 20
Min no. of reply  5
Max no. of reply  22

 ------START: 2021_12: CNCC_0621 ------- 

Eng lang replies  13212
Greater than equal to 5 : 557
Min no. of reply  5
Max no. of reply  452

 ------START: 2021_12: MX_0621 ------- 

Eng lang replies  5577
Greater than equal to 5 : 31
Min no. of reply  5
Max no. of reply  1470

 ------START: 2021_12: REA_0621 ------- 

Eng lang replies  878
Greater than equal to 5 : 3
Min no. of reply  54
Max no. of reply  643

 ------START: 2021_12: RNA_0621 ------- 

Eng lang replies  3871
Greater than equal to 5 : 89
Min no. of reply  5
Max no. of reply  228

 ------START: 2021_12: Tanzania_0621 ------- 

Eng lang replies  12781
Greater than equal to 5 : 302
Min no. of reply  5
Max no. of reply  629

 ------START: 2021_12: uganda_0621 ------- 

Eng lang replies  137205
Greater than equal to 5 : 4366
Min no. of reply  5
Max no. of reply  3539

 ------START: 2021_12: 